In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, matthews_corrcoef

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
EC=pd.read_csv('/content/drive/MyDrive/MScProject/online_shoppers_intention.csv')
EC.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [5]:
variable=['Weekend','Revenue']
for col in variable:
    encoder = LabelEncoder()
    encoder.fit(EC[col])
    print('Column:', col)
    print('Original categories:', encoder.classes_)
    print('Encoded values:', encoder.transform(encoder.classes_))
    print('\n')
    EC[col] = encoder.fit_transform(EC[col])

Column: Weekend
Original categories: [False  True]
Encoded values: [0 1]


Column: Revenue
Original categories: [False  True]
Encoded values: [0 1]




In [6]:
EC['Month'] = EC['Month'].map({'Feb': 2, 'Mar': 3, 'May': 5,'June':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})

In [7]:
MMC = ['Administrative_Duration','Informational_Duration','ProductRelated_Duration','BounceRates','ExitRates','PageValues','SpecialDay']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
EC[MMC] = scaler.fit_transform(EC[MMC])

In [8]:

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
#encoder.fit(df['category'])
# Fit and transform the categorical variable
EC['VisitorType'] = label_encoder.fit_transform(EC['VisitorType'])

EC.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,1.00,1.00,0.0,0.0,2,1,1,1,1,2,0,0
1,0,0.0,0,0.0,2,0.001000,0.00,0.50,0.0,0.0,2,2,2,1,2,2,0,0
2,0,0.0,0,0.0,1,0.000000,1.00,1.00,0.0,0.0,2,4,1,9,3,2,0,0
3,0,0.0,0,0.0,2,0.000042,0.25,0.70,0.0,0.0,2,3,2,2,4,2,0,0
4,0,0.0,0,0.0,10,0.009809,0.10,0.25,0.0,0.0,2,3,3,1,4,2,1,0


In [9]:
X = EC.drop('Revenue', axis=1)
y = EC['Revenue']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model on the training dataset
dt_classifier.fit(X_train, y_train)


# Make predictions on the test dataset
y_pred = dt_classifier.predict(X_test)
print(classification_report(y_test, y_pred))
# Make predictions on the testing set
y_pred_proba = dt_classifier.predict_proba(X_test)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_test, y_pred)
print("MCC Score:", mcc)


              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2055
           1       0.56      0.57      0.57       411

    accuracy                           0.85      2466
   macro avg       0.74      0.74      0.74      2466
weighted avg       0.86      0.85      0.85      2466

AUC Score: 0.7411192214111921
MCC Score: 0.478544930106141


In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X, y)
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)
dt_classifier.fit(X_trainn, y_trainn)
y_pred = dt_classifier.predict(X_testt)
print(classification_report(y_testt, y_pred))

# Make predictions on the testing set
y_pred_proba = dt_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)

              precision    recall  f1-score   support

           0       0.92      0.89      0.90      2062
           1       0.90      0.92      0.91      2107

    accuracy                           0.90      4169
   macro avg       0.91      0.90      0.90      4169
weighted avg       0.91      0.90      0.90      4169

AUC Score: 0.9046146349727043
MCC Score: 0.8097712702943418


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Initialize the Random Forest classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_trainn, y_trainn)

# Get the best parameters and the best accuracy score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy Score:", best_score)
dt_classifier = DecisionTreeClassifier(**best_params, random_state=42)

# Train the model on the entire training dataset (X_train and y_train combined)
dt_classifier.fit(X_trainn, y_trainn)

# Make predictions on the test dataset (X_test)
y_pred = dt_classifier.predict(X_testt)
print(classification_report(y_testt, y_pred))
# Make predictions on the testing set
y_pred_proba = dt_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)


# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)


Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best Accuracy Score: 0.9067466266866567
              precision    recall  f1-score   support

           0       0.92      0.91      0.91      2062
           1       0.91      0.92      0.92      2107

    accuracy                           0.92      4169
   macro avg       0.92      0.92      0.92      4169
weighted avg       0.92      0.92      0.92      4169

MCC Score: 0.830672943228811
AUC Score: 0.948805814252708


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the k-NN classifier
knn_classifier = KNeighborsClassifier()

# Train the model on the training dataset
knn_classifier.fit(X_train, y_train)

# Make predictions on the test dataset
y_pred = knn_classifier.predict(X_test)




print(classification_report(y_test, y_pred))
# Make predictions on the testing set
y_pred_proba = dt_classifier.predict_proba(X_test)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_test, y_pred)
print("MCC Score:", mcc)

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      2055
           1       0.29      0.09      0.13       411

    accuracy                           0.81      2466
   macro avg       0.57      0.52      0.51      2466
weighted avg       0.75      0.81      0.77      2466

AUC Score: 0.9472037224501395
MCC Score: 0.07700631910619589


In [25]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X, y)
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)
knn_classifier.fit(X_trainn, y_trainn)
y_pred = knn_classifier.predict(X_testt)
print(classification_report(y_testt, y_pred,zero_division=1))

# Make predictions on the testing set
y_pred_proba = knn_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)

              precision    recall  f1-score   support

           0       0.95      0.68      0.79      2062
           1       0.75      0.96      0.85      2107

    accuracy                           0.82      4169
   macro avg       0.85      0.82      0.82      4169
weighted avg       0.85      0.82      0.82      4169

AUC Score: 0.905369589245032
MCC Score: 0.6697263575177452


In [35]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Initialize the k-NN classifier
knn_classifier = KNeighborsClassifier()

# Initialize Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_trainn, y_trainn)

# Get the best parameters and the best accuracy score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Accuracy Score:", best_score)

# Initialize a new k-NN classifier with the best parameters
best_knn_classifier = KNeighborsClassifier(**best_params)

# Train the model on the entire training dataset with the best parameters
best_knn_classifier.fit(X_trainn, y_trainn)

# Make predictions on the test dataset
y_pred = best_knn_classifier.predict(X_testt)

# Evaluate the model's performance
accuracy = accuracy_score(y_testt, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
print("Classification Report:")
print(classification_report(y_testt, y_pred))
y_pred_proba = best_knn_classifier.predict_proba(X_testt)[:, 1]  # Predict probabilities for class 1

# Calculate MCC score
mcc = matthews_corrcoef(y_testt, y_pred)
print("MCC Score:", mcc)


# Calculate AUC score
auc = roc_auc_score(y_testt, y_pred_proba)
print("AUC Score:", auc)

Best Parameters: {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'}
Best Accuracy Score: 0.8214092953523238
Accuracy: 0.8438474454305589
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.72      0.82      2062
           1       0.78      0.96      0.86      2107

    accuracy                           0.84      4169
   macro avg       0.86      0.84      0.84      4169
weighted avg       0.86      0.84      0.84      4169

MCC Score: 0.7070266953558311
AUC Score: 0.9129276482207709
